# Leonardo's Armored Walker: A Renaissance Tank

> *"I will make covered chariots, safe and unattackable..."*  
> — Leonardo da Vinci, Letter to Ludovico Sforza

## Introduction
Leonardo designed an armored vehicle that could move in any direction. While his famous sketch shows wheels, he also explored walking mechanisms. This notebook simulates a **walking tank** that combines the self-propelled cart's drive with the mechanical lion's legs.

We will analyze:
1.  **Power Requirements**: Moving a heavy armored shell.
2.  **Gait Timing**: Coordinating legs to maintain momentum.
3.  **Energy Efficiency**: Spring power vs. friction and drag.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Power Balance
The spring provides power $P_{in}$ which must overcome resistance $P_{out}$:
$$ P_{in} = \tau_{spring} \cdot \omega_{cam} \cdot \eta $$
$$ P_{out} = (F_{drag} + F_{friction}) \cdot v $$

### 2. Resistance Forces
- **Rolling/Walking Resistance**: $F_{friction} = \mu \cdot m \cdot g$
- **Aerodynamic Drag**: $F_{drag} = \frac{1}{2} \rho C_d A v^2$


In [ ]:
def simulate_walker(mass_kg, gear_ratio, spring_k):
    dt = 0.1
    time = np.arange(0, 60, dt)
    
    # Parameters
    mu = 0.1 # High friction for walking mechanism
    g = 9.81
    drag_coeff = 1.2
    area = 3.0 # Large frontal area
    rho = 1.225
    
    # Initial State
    theta_spring = 100.0 # Initial wind (radians)
    velocity = 0.0
    position = 0.0
    
    pos_hist = []
    vel_hist = []
    power_avail_hist = []
    power_req_hist = []
    
    for t in time:
        # Available Power
        torque = spring_k * theta_spring if theta_spring > 0 else 0
        # Assume constant cam speed target (regulated by escapement)
        target_v = 1.0 # m/s
        omega_cam = target_v / 0.5 # stride length 0.5m
        
        # Power from spring through gears
        power_in = torque * (omega_cam / gear_ratio) * 0.8 # 80% efficiency
        
        # Required Power
        friction = mu * mass_kg * g
        drag = 0.5 * rho * drag_coeff * area * velocity**2
        force_req = friction + drag
        power_req = force_req * velocity if velocity > 0 else force_req * 0.1 # Start-up power
        
        # Dynamics
        if power_in > power_req:
            # Excess power accelerates (limited by escapement)
            net_force = (power_in / max(velocity, 0.1)) - force_req
            accel = net_force / mass_kg
            # Escapement limits max speed
            if velocity >= target_v:
                accel = min(accel, 0)
        else:
            # Deficit slows down
            net_force = -force_req
            accel = net_force / mass_kg
            
        velocity += accel * dt
        if velocity < 0: velocity = 0
        position += velocity * dt
        
        # Unwind Spring
        power_used = min(power_in, power_req) if velocity > 0 else 0
        # Energy = Power * dt
        d_energy = power_used * dt
        # E = 0.5 k theta^2 -> dE = k theta dtheta
        if theta_spring > 0:
            d_theta = d_energy / (spring_k * theta_spring)
            theta_spring -= d_theta
            
        pos_hist.append(position)
        vel_hist.append(velocity)
        power_avail_hist.append(power_in)
        power_req_hist.append(power_req)
        
    return time, pos_hist, vel_hist, power_avail_hist, power_req_hist

def plot_walker(mass, ratio, k):
    t, pos, vel, p_avail, p_req = simulate_walker(mass, ratio, k)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Motion
    ax1.plot(t, vel, 'k-', label='Velocity (m/s)')
    ax1_twin = ax1.twinx()
    ax1_twin.plot(t, pos, 'b--', label='Distance (m)')
    ax1.set_title('Walker Motion')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Velocity (m/s)')
    ax1_twin.set_ylabel('Distance (m)', color='b')
    
    # Plot 2: Power
    ax2.plot(t, p_avail, 'g-', label='Power Available')
    ax2.plot(t, p_req, 'r--', label='Power Required')
    ax2.set_title('Power Balance')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Power (W)')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"Total Distance: {pos[-1]:.1f} m")
    if pos[-1] < 1.0:
        print("STALLED: Not enough power to move!")

interact(plot_walker, 
         mass=widgets.IntSlider(min=500, max=2000, step=100, value=1000, description='Mass (kg)'),
         ratio=widgets.FloatSlider(min=10.0, max=50.0, step=5.0, value=20.0, description='Gear Ratio'),
         k=widgets.IntSlider(min=100, max=1000, step=50, value=500, description='Spring K'));

## Conclusion

The simulation highlights the massive power requirements of an armored walker. Unlike a wheeled cart, a walker has high internal friction. Leonardo's spring drive would likely have struggled to move a full-scale armored version for any significant distance, suggesting it was more of a conceptual prototype or theatrical piece.